# Analysis of Vienna Night Run Results

## Data Cleaning

In [63]:
import pandas as pd
import numpy as np

Read raw data and add column names.

In [64]:
year = 2022
path_to_raw = "data/raw/vnr-results-public-" + str(year) + ".txt"
raw_data = pd.read_csv(path_to_raw, delimiter='\t', header=None, na_values=" ")
raw_data.drop(axis=1, columns=[0, 8], inplace=True)
raw_data.columns = ["id", "name", "sex", "company", "mins_per_km", "km_per_hour", "time"]
z=raw_data

In [65]:
z.head()

,id,name,sex,company,mins_per_km,km_per_hour,time
0,2979,Dominik Stadlmann,m,Tony's Laufshop,2:52,20.8,"14:23,6"
1,4178,Dominik Jandl,m,adesso,2:54,20.6,"14:31,5"
2,2342,Jan Michael Ratay,m,LCA Umdasch Amstetten,3:00,19.9,"15:00,6"
3,3218,Indrek Tobreluts,m,Spordiklubi TT,3:02,19.7,"15:10,5"
4,2952,Philipp Gintenstorfer,m,Hawi'd Ehre,3:03,19.6,"15:17,4"


Calculate time attributes.

In [66]:
# Keep only times less than 60 min
z = z.loc[z["time"].str.split(":").str.len() == 2].copy()
# z.head()
# Split minutes part from the rest
time_split = z["time"].str.split(":")
# Populate minutes column
z.loc[:, "mins"] = [int(x[0]) for x in time_split]
z.loc[:, "rest"] = [x[1] for x in time_split]
# Split seconds from the tenths of second
secs_split = z["rest"].str.split(",")
# Populate seconds column
z.loc[:, "secs"] = [int(x[0]) for x in secs_split]
# z["secs"] = pd.Series([x[0] for x in secs_split]).astype(int)
# Populate tenth of seconds column
z.loc[:, "tsecs"] = [int(x[1]) for x in secs_split]
# z["tsecs"] = pd.Series([x[1] for x in secs_split]).astype(int)
# Calculate total time in seconds
z["time_secs"] = z["mins"] * 60 + z["secs"] + z["tsecs"] * 0.1
# Calculate total time in minutes
z["time_mins"] = z["time_secs"] / 60
# Calculate pace (seconds per km)
z["secs_per_km_calc"] = z["time_secs"] / 5
z.head()

,id,name,sex,company,mins_per_km,km_per_hour,time,mins,rest,secs,tsecs,time_secs,time_mins,secs_per_km_calc
0,2979,Dominik Stadlmann,m,Tony's Laufshop,2:52,20.8,"14:23,6",14,"23,6",23,6,863.6,14.393333,172.72
1,4178,Dominik Jandl,m,adesso,2:54,20.6,"14:31,5",14,"31,5",31,5,871.5,14.525000,174.30
2,2342,Jan Michael Ratay,m,LCA Umdasch Amstetten,3:00,19.9,"15:00,6",15,"00,6",0,6,900.6,15.010000,180.12
3,3218,Indrek Tobreluts,m,Spordiklubi TT,3:02,19.7,"15:10,5",15,"10,5",10,5,910.5,15.175000,182.10
4,2952,Philipp Gintenstorfer,m,Hawi'd Ehre,3:03,19.6,"15:17,4",15,"17,4",17,4,917.4,15.290000,183.48


Fix categorical factors.

In [67]:
# Fix sex
z["sex"] = np.select(
    [z["sex"] == "m", z["sex"] == "w"],
    ["m", "f"],
    default=pd.NA
)
z["sex"] = z["sex"].astype("category")
z.head()

,id,name,sex,company,mins_per_km,km_per_hour,time,mins,rest,secs,tsecs,time_secs,time_mins,secs_per_km_calc
0,2979,Dominik Stadlmann,m,Tony's Laufshop,2:52,20.8,"14:23,6",14,"23,6",23,6,863.6,14.393333,172.72
1,4178,Dominik Jandl,m,adesso,2:54,20.6,"14:31,5",14,"31,5",31,5,871.5,14.525000,174.30
2,2342,Jan Michael Ratay,m,LCA Umdasch Amstetten,3:00,19.9,"15:00,6",15,"00,6",0,6,900.6,15.010000,180.12
3,3218,Indrek Tobreluts,m,Spordiklubi TT,3:02,19.7,"15:10,5",15,"10,5",10,5,910.5,15.175000,182.10
4,2952,Philipp Gintenstorfer,m,Hawi'd Ehre,3:03,19.6,"15:17,4",15,"17,4",17,4,917.4,15.290000,183.48


Confirm pace as (incorrectly) provided in the original dataset.

In [68]:
# Round the calculated seconds per km to the lowest integer. This is 
# technically incorrect, but it's how it's handled in the original data!
z["secs_per_km_calc_floor"] = np.floor(z["secs_per_km_calc"])
z["pace_secs"] = z["secs_per_km_calc_floor"] % 60
pace_split = z["mins_per_km"].str.split(":")
z.loc[:, "pace_secs_orig"] = [int(x[1]) for x in pace_split]
z.head()

,id,name,sex,company,mins_per_km,km_per_hour,time,mins,rest,secs,tsecs,time_secs,time_mins,secs_per_km_calc,secs_per_km_calc_floor,pace_secs,pace_secs_orig
0,2979,Dominik Stadlmann,m,Tony's Laufshop,2:52,20.8,"14:23,6",14,"23,6",23,6,863.6,14.393333,172.72,172.0,52.0,52
1,4178,Dominik Jandl,m,adesso,2:54,20.6,"14:31,5",14,"31,5",31,5,871.5,14.525000,174.30,174.0,54.0,54
2,2342,Jan Michael Ratay,m,LCA Umdasch Amstetten,3:00,19.9,"15:00,6",15,"00,6",0,6,900.6,15.010000,180.12,180.0,0.0,0
3,3218,Indrek Tobreluts,m,Spordiklubi TT,3:02,19.7,"15:10,5",15,"10,5",10,5,910.5,15.175000,182.10,182.0,2.0,2
4,2952,Philipp Gintenstorfer,m,Hawi'd Ehre,3:03,19.6,"15:17,4",15,"17,4",17,4,917.4,15.290000,183.48,183.0,3.0,3


Calculate correct speed.

Keep only relevant columns.

Save cleaned results.